<a href="https://colab.research.google.com/github/Saultr21/IA-Y-BIGDATA/blob/main/M2D/Whisper/Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación de Librerías y Dependencias Básicas

En este bloque se instalan todas las dependencias necesarias para el proyecto:

- **Transformers:** Para utilizar modelos de Hugging Face.
- **Torch:** Base de PyTorch para operaciones de deep learning.
- **FFmpeg:** Para la manipulación y conversión de archivos de audio.
- **Datasets:** Librería auxiliar para el manejo de datos.
- **yt-dlp:** Para descargar y extraer audio (por ejemplo, de YouTube).

In [ ]:
# Instalamos las librerías necesarias
!pip install transformers  # Para usar modelos de Hugging Face
!pip install torch         # Librería base de PyTorch
!apt-get install ffmpeg    # Para manipular y convertir archivos de audio
!pip install datasets      # Librería auxiliar para manejo de datos
!pip install yt-dlp
!apt-get install ffmpeg -y

# Instalación del Modelo Whisper de OpenAI

En este bloque se instala la versión más reciente de **Whisper** directamente desde el repositorio oficial de GitHub de OpenAI. Esto garantiza que se cuente con la última versión disponible para la transcripción de audio.


In [ ]:
!pip install git+https://github.com/openai/whisper.git  # Instala Whisper de OpenAI

# Importación de Librerías

En este bloque se importan los módulos necesarios para:

- **whisper:** Cargar y utilizar el modelo de transcripción.
- **os:** Interactuar con el sistema operativo (gestión de archivos, etc.).
- **language_tool_python:** Posible uso en corrección gramatical o análisis del lenguaje.
- **files (de google.colab):** Permitir la subida de archivos locales.


In [6]:
import whisper
import os
import language_tool_python
from google.colab import files

# Selección del Método de Carga del Audio

Este bloque interactúa con el usuario para elegir cómo obtener el audio a transcribir. Se ofrecen dos opciones:

1. **Subir un archivo local (WAV o MP3):**
   - El usuario sube un archivo.
   - Si el archivo es WAV, se convierte a MP3 usando FFmpeg.
   - Si el archivo es MP3, se renombra a `audio.mp3`.

2. **Descargar audio desde YouTube:**
   - Se solicita la URL de un video de YouTube.
   - Con *yt-dlp* se descarga y extrae el audio en formato MP3.
   - Se renombra el archivo descargado a `audio.mp3`.

Si el usuario ingresa una opción no válida, se muestra un mensaje de error.


In [ ]:
# =========================
# SELECCIÓN DE MÉTODO DE CARGA
# =========================
print("Seleccione la opción:")
print("1: Subir un archivo local (WAV o MP3)")
print("2: Descargar audio de YouTube")
option = input("Ingrese 1 o 2: ")

if option.strip() == '1':
    # -----------------------------------
    # SUBIDA Y CONVERSIÓN DE ARCHIVO LOCAL
    # -----------------------------------
    print("Por favor, sube tu archivo de audio (WAV o MP3).")
    uploaded = files.upload()
    # Se extrae el nombre del primer archivo subido
    filename = list(uploaded.keys())[0]

    if filename.lower().endswith('.wav'):
        print("El archivo subido es WAV. Convirtiendo a MP3...")
        !ffmpeg -i "{filename}" -vn -ar 44100 -ac 2 -b:a 192k "audio.mp3"
        print("Conversión completa. Archivo guardado como audio.mp3")
    elif filename.lower().endswith('.mp3'):
        print("El archivo subido es MP3. No se requiere conversión.")
        !mv "{filename}" "audio.mp3"
        print("Archivo renombrado a audio.mp3")
    else:
        print("Formato no reconocido (no es WAV ni MP3). Se mantiene el nombre original.")
        print(f"Archivo: {filename}")

elif option.strip() == '2':
    # -----------------------------------
    # DESCARGAR AUDIO DESDE YOUTUBE CON yt-dlp
    # -----------------------------------
    video_url = input("Ingresa la URL del video de YouTube: ")
    output_name = "audio.mp3"

    print("Descargando y convirtiendo audio desde YouTube...")
    # Se descarga y extrae el audio en formato MP3
    !yt-dlp -x --audio-format mp3 -o "{output_name}.%(ext)s" {video_url}

    # Aseguramos que el archivo final se llame 'audio.mp3'
    !mv "{output_name}.mp3" "audio.mp3"

    print("Descarga y conversión completadas. Archivo guardado como audio.mp3")
else:
    print("Opción no válida. Por favor, reinicie el proceso y seleccione 1 o 2.")


# Carga del Modelo y Transcripción del Audio

En este bloque se realiza el procesamiento principal:

1. **Carga del Modelo:**  
   Se carga la versión "base" del modelo Whisper de OpenAI para la transcripción.

2. **Transcripción:**  
   Se utiliza el método `transcribe` para convertir el archivo `audio.mp3` en texto, mostrando información en tiempo real gracias al modo verbose.

3. **Guardar Resultado:**  
   La transcripción se guarda en un archivo llamado `transcripcion.txt` para su posterior consulta.


In [ ]:
# Carga del modelo
model = whisper.load_model("base")

# Transcripción con salida en tiempo real (modo verbose)
result = model.transcribe("audio.mp3", verbose=True)

# Guardamos la transcripción final en un archivo .txt
with open("transcripcion.txt", "w", encoding="utf-8") as f:
    f.write(result["text"])